In [10]:
#LENET_SLR_A
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Model,Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils

fc_id = 5 # FC Layer Number
rank = 6
sr = 0.5
rr = 0.5

# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

json_file = open('Lenet.json', 'r')
lenet_model_json = json_file.read()
json_file.close()
lenet_model = model_from_json(lenet_model_json)
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

def sparse_SVD_ar(U,S,V,inp_act,out_act,keep,sr,rr):
    
    tU, tS, tV = U[:, 0:keep], S[0:keep], V[0:keep, :]

    # Input node selection
    iwm = np.sum(abs(inp_act),axis=0)
    imid = sorted(iwm)[int(U.shape[0]*sr)]
    ipl = np.where(iwm<imid)[0]

    # Output node selection
    owm = np.sum(abs(out_act),axis=0)
    omid = sorted(owm)[int(V.shape[1]*sr)]
    opl = np.where(owm<omid)[0]
    
    # Masking the weights
    subrank = int(keep*rr)
    for ind in ipl:
        tU[ind,subrank:]=0

    for ind in opl:
        tV[subrank:,ind]=0

    return tU, tS, tV

# Loading Model
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
act_fc1 = Model(input=lenet_model.input, output=lenet_model.get_layer('flatten_2').output)
act_fc2 = Model(input=lenet_model.input, output=lenet_model.get_layer('dense_4').output)

# Flatten 
inp_act = act_fc1.predict(x_train)
out_act = act_fc2.predict(x_train)

keep = rank

fc1 = lenet_model.layers[fc_id].get_weights()

weights = np.copy(fc1)

# Decomposition and Reconstruction
U, S, V = np.linalg.svd(weights[0], full_matrices=False)
tU, tS, tV = sparse_SVD_ar(U,S,V,inp_act,out_act,keep,sr,rr)

weights_t = np.matmul(np.matmul(tU, np.diag(tS)), tV)

# Loading weights for new model
weights[0] = weights_t
lenet_model.layers[fc_id].set_weights(weights)

# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
slr_accuracy = score[1]
print('Params:',tU.size+tS.size+tV.size,', Acc:', slr_accuracy)

C:\Users\Will\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("fl...)`
C:\Users\Will\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("de...)`


Params: 3126 , Acc: 0.6118000149726868


In [11]:
#LENET_SLR_C
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Model,Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils

fc_id = 5 # FC Layer Number
rank = 6
sr = 0.5
rr = 0.5

# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

json_file = open('Lenet.json', 'r')
lenet_model_json = json_file.read()
json_file.close()
lenet_model = model_from_json(lenet_model_json)
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])


keep = rank
fc1 = lenet_model.layers[fc_id].get_weights()

weights = fc1[0]

ip_node = weights.shape[0]
op_node = weights.shape[1]

U, S, V = np.linalg.svd(weights, full_matrices=False)
tU, tS, tV = U[:, 0:keep], S[0:keep], V[0:keep, :]

fc1[0] = np.matmul(np.matmul(tU, np.diag(tS)), tV)
lenet_model.layers[fc_id].set_weights(fc1)

score = lenet_model.evaluate(x_train, y_train, verbose=0)
svd_cost = score[0]

subrank = int(keep*rr)

imc = np.zeros([ip_node])

for ind in range(0,ip_node):
    tempU = np.copy(tU)
    tempU[ind,subrank:]=0
    weights_t = np.copy(fc1)
    weights_t[0] = np.matmul(np.matmul(tempU, np.diag(tS)), tV)
    lenet_model.layers[fc_id].set_weights(weights_t)
    score = lenet_model.evaluate(x_train, y_train, verbose=0)
    svd_cost_t = score[0]
    imc[ind] = abs(svd_cost-svd_cost_t)
    lenet_model.layers[fc_id].set_weights(fc1)
    print("In",ind)
    
omc = np.zeros([op_node])

for ind in range(0,op_node):
    temptV = np.copy(tV)
    temptV[subrank:,ind]=0
    weights_t = np.copy(fc1)
    weights_t[0] = np.matmul(np.matmul(tU, np.diag(tS)), temptV)
    lenet_model.layers[fc_id].set_weights(weights_t)
    score = lenet_model.evaluate(x_train, y_train, verbose=0)
    svd_cost_t = score[0]
    omc[ind] = abs(svd_cost-svd_cost_t)
    lenet_model.layers[fc_id].set_weights(fc1)
    print("Out",ind)
    
# Loading Model
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Loading weights of the model
fc1 = lenet_model.layers[fc_id].get_weights()

weights = np.copy(fc1)
U, S, V = np.linalg.svd(weights[0], full_matrices=False)
tU, tS, tV = U[:, 0:keep], S[0:keep], V[0:keep, :]

# Input node selection
imid = sorted(imc)[int(ip_node*sr)]
ipl = np.where(imc<imid)[0]

# Output node selection
omid = sorted(omc)[int(op_node*sr)]
opl = np.where(omc<omid)[0]

# Masking the weights
subrank = int(keep*rr)
for ind in ipl:
    tU[ind,subrank:]=0

for ind in opl:
    tV[subrank:,ind]=0
    
weights_t = np.matmul(np.matmul(tU, np.diag(tS)), tV)

# Loading weights for new model
weights[0] = weights_t
lenet_model.layers[fc_id].set_weights(weights)

# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
slr_accuracy = score[1]
print('Params:',tU.size+tS.size+tV.size,', Acc:', slr_accuracy)

In 0
In 1
In 2
In 3
In 4
In 5
In 6
In 7
In 8
In 9
In 10
In 11
In 12
In 13
In 14
In 15
In 16
In 17
In 18
In 19
In 20
In 21
In 22
In 23
In 24
In 25
In 26
In 27
In 28
In 29
In 30
In 31
In 32
In 33
In 34
In 35
In 36
In 37
In 38
In 39
In 40
In 41
In 42
In 43
In 44
In 45
In 46
In 47
In 48
In 49
In 50
In 51
In 52
In 53
In 54
In 55
In 56
In 57
In 58
In 59
In 60
In 61
In 62
In 63
In 64
In 65
In 66
In 67
In 68
In 69
In 70
In 71
In 72
In 73
In 74
In 75
In 76
In 77
In 78
In 79
In 80
In 81
In 82
In 83
In 84
In 85
In 86
In 87
In 88
In 89
In 90
In 91
In 92
In 93
In 94
In 95
In 96
In 97
In 98
In 99
In 100
In 101
In 102
In 103
In 104
In 105
In 106
In 107
In 108
In 109
In 110
In 111
In 112
In 113
In 114
In 115
In 116
In 117
In 118
In 119
In 120
In 121
In 122
In 123
In 124
In 125
In 126
In 127
In 128
In 129
In 130
In 131
In 132
In 133
In 134
In 135
In 136
In 137
In 138
In 139
In 140
In 141
In 142
In 143
In 144
In 145
In 146
In 147
In 148
In 149
In 150
In 151
In 152
In 153
In 154
In 155
In 156
In 157
In 1

In [12]:
#LENET_SLR_W
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Model,Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils

fc_id = 5 # FC Layer Number
rank = 6
sr = 0.5
rr = 0.5

# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

json_file = open('Lenet.json', 'r')
lenet_model_json = json_file.read()
json_file.close()
lenet_model = model_from_json(lenet_model_json)
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
print("lenet model from disk")

def sparse_SVD_wr(weights,U,S,V,keep,rate,subrank_rate):
    tU, tS, tV = U[:, 0:keep], S[0:keep], V[0:keep, :]

    # Input node selection
    iwm = np.sum(abs(weights),axis=1)
    imid = sorted(iwm)[int(weights.shape[0]*rate)]
    ipl = np.where(iwm<imid)[0]

    # Output node selection
    owm = np.sum(abs(weights),axis=0)
    omid = sorted(owm)[int(weights.shape[1]*rate)]
    opl = np.where(owm<omid)[0]

    # Masking the weights
    subrank = int(keep*subrank_rate)
    for ind in ipl:
        tU[ind,subrank:]=0

    for ind in opl:
        tV[subrank:,ind]=0

    return tU, tS, tV


keep = rank
fc1 = lenet_model.layers[fc_id].get_weights()

weights = np.copy(fc1)

# Decomposition and Reconstruction
U, S, V = np.linalg.svd(weights[0], full_matrices=False)
tU, tS, tV = sparse_SVD_wr(weights[0],U,S,V,keep,sr,rr)

weights_t = np.matmul(np.matmul(tU, np.diag(tS)), tV)

# Loading weights for new model
weights[0] = weights_t
lenet_model.layers[fc_id].set_weights(weights)

# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
slr_accuracy = score[1]
print('Params:',tU.size+tS.size+tV.size,', Acc:', slr_accuracy)

lenet model from disk
Params: 3126 , Acc: 0.6793000102043152


fc1 shape (400, 120)
(400, 120)
(120,)
(120, 120)
(400, 8)
(8,)
(8, 120)
(400, 120)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_3 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_4 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               48120     
____________________________________

In [3]:
#LENET_TRAIN
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils


# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

#Instantiate an empty model
model = Sequential()

# C1 Convolutional Layer
model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32,32,1)))

# S2 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C3 Convolutional Layer
model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# S4 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

#Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())

# C5 Fully Connected Convolutional Layer
model.add(layers.Dense(120, activation='tanh'))


#Output Layer with softmax activation
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='Adam', metrics=["accuracy"]) 
model.summary()


hist = model.fit(x=x_train,y=y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose=1) 

# Write the testing input and output variables
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


# serialize model to JSON
model_json = model.to_json()

# Write the file name of the model

with open("Lenet.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
# Write the file name of the weights

model.save_weights("Lenet.h5")
print("Saved model to disk")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_3 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_4 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

In [82]:
#Lenet Compressed SVD
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Model,Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils


fc_id = 5 # FC Layer Number
rank = 8 # truncate value


# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

json_file = open('Lenet.json', 'r')
lenet_model_json = json_file.read()
json_file.close()
lenet_model = model_from_json(lenet_model_json)
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

lenet_model.summary()

# Loading weights of the model
keep = rank
fc1 = lenet_model.layers[fc_id].get_weights()
fc2 = lenet_model.layers[fc_id+1].get_weights()
lenet_model.pop()
lenet_model.pop()
lenet_model.add(layers.Dense(rank, activation = 'tanh'))
lenet_model.add(layers.Dense(120))
lenet_model.add(layers.Dense(10, activation = 'softmax'))

lenet_model.summary()
# Decomposition and Reconstruction

OU, OS, OV = np.linalg.svd(fc1[0], full_matrices=False)
U, S, V = np.linalg.svd(fc1[0], full_matrices=True)

print('Shapes False', OU.shape, OS.shape, OV.shape)
print('Shapes True', U.shape, S.shape, V.shape)

fc1_n = np.matmul(np.matmul(OU, np.diag(OS)), OV)
tU, tS, tV = OU[:, 0:keep], OS[0:keep], OV[0:keep, :]
fc1_t = np.matmul(np.matmul(tU, np.diag(tS)), tV)

S = np.diag(S)
TU, TS, TV =  U[:, 0:keep], S[0:keep, 0:keep], V[0:keep,:]
print('Shapes', TU.shape, TS.shape, TV.shape)


svdlayer2 = np.matmul(TS, TV)
svdlayer1 = TU

print('svdlayer1 shape', svdlayer1.shape)
print('svdlayer2 shape', svdlayer2.shape)
# Loading weights for new model
#fc1[0] = fc1_t
#lenet_model.layers[fc_id].set_weights(fc1)

lenet_model.layers[fc_id].set_weights(svdlayer1)
lenet_model.layers[6].set_weights(svdlayer2)
lenet_model.layers[7].set_weights(fc2)

lenet_model.summary()

# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
truncsvd_accuracy = score[1]
print(tU.size+tS.size+tV.size,' ', truncsvd_accuracy)

#save weights in necessary right side layers
#pop until we get to layer we need
#add svd layers
#add back saved layers
#retest



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_3 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_4 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

ValueError: You called `set_weights(weights)` on layer "dense_147" with a weight list of length 400, but the layer was expecting 2 weights. Provided weights: [[ 0.00428926 -0.0257377   0.00048366 ...  0.01284...

In [83]:
#LENET_SVD
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Model,Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils


fc_id = 5 # FC Layer Number
rank = 8

# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

json_file = open('Lenet.json', 'r')
lenet_model_json = json_file.read()
json_file.close()
lenet_model = model_from_json(lenet_model_json)
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])



# Loading weights of the model
keep = rank
fc1 = lenet_model.layers[fc_id].get_weights()

print('fc1 shape',fc1[0].shape)
# Decomposition and Reconstruction
U, S, V = np.linalg.svd(fc1[0], full_matrices=False)
tU, tS, tV = U[:, 0:keep], S[0:keep], V[0:keep, :]
fc1_t = np.matmul(np.matmul(tU, np.diag(tS)), tV)

lenet_model.summary()

print(np.count_nonzero(fc1_t))

# Loading weights for new model
fc1[0] = fc1_t
lenet_model.layers[fc_id].set_weights(fc1)


flatten_layer = lenet_model.layers[5].get_weights()
print(flatten_layer)

# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
truncsvd_accuracy = score[1]
print(tU.size+tS.size+tV.size,' ', truncsvd_accuracy)

fc1 shape (400, 120)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_3 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_4 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_4 (Dense)              (Non